In [5]:
import requests
import pandas as pd
import json
import pyodbc
from datetime import date
from datetime import datetime
from time import sleep
today = date.today()
now = datetime.now()
print(today,now)

2019-11-06 2019-11-06 16:28:19.271323


In [6]:
urls = ['http://api.openweathermap.org/data/2.5/weather?q=San%20Francisco,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=San%20Jose,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=New%20York,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Boston,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Los%20Angeles,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Las%20Vegas,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Minneapolis,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Detroit,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Chicago,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Seattle,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Phoenix,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Miami,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Portland,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Atlanta,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Honolulu,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Juneau,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Houston,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Helena,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Pittsburgh,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Denver,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374',
'http://api.openweathermap.org/data/2.5/weather?q=Boulder,US&units=imperial&APPID=de2e3055b593558ad302dda095e9f374']

cityWeather = []
for row in urls:
    response = requests.get(row)
    responseDict = response.json()
    responseJSON = json.dumps(responseDict)
    data = json.loads(responseJSON)
    cityWeather.append(data)

In [7]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=LAPTOP-RC2ANL8F\SQLEXPRESS;'
                      'Database=WeatherWars;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#cityWeather[0]['main']['temp']
#cityWeather[0]['rain']['1h']
#cityWeather[0]['wind']['speed']
#cityWeather[0]['name']
#cursor.execute("select cName, geoId, cState from cities where cName = 'San Francisco'")
type(cityWeather[0]['main']['temp'])

float

In [8]:
for row in cityWeather:
    temp = row['main']['temp']
    try:
        row['rain']['1h']
        rain = row['rain']['1h']
    except:
        rain = 0
    wind = row['wind']['speed']
    name = row['name']
    today = date.today()
    now = datetime.now()
    cursor.execute("select tHigh, tLow, rainLevel, windSpeed from weather where cName = ? and currentDate = ?",(name, today))
    values = cursor.fetchone()
    if values != None:
        high = values[0]
        low = values[1]
        rainL = values[2]
        windS = values[3]
        
        if temp > high:
            cursor.execute("update weather set tHigh = ?, lastUpdated = ? where cName = ? and currentDate = ?", (temp, now, name, today))
        if temp < low:
            cursor.execute("update weather set tLow = ?, lastUpdated = ? where cName = ? and currentDate = ?", (temp, now, name, today))
        if rain > rainL:
            cursor.execute("update weather set rainLevel = ?, lastUpdated = ? where cName = ? and currentDate = ?", (rain, now, name, today))
        if wind > windS:
            cursor.execute("update weather set windSpeed = ?, lastUpdated = ? where cName = ? and currentDate = ?", (wind, now, name, today))
    else:
        cursor.execute("insert into weather values(?,?,?,?,?,?,?)",(name, now, today, temp, temp, rain, wind))
conn.commit()

In [9]:
cursor.execute("select * from weather")


In [10]:
for row in cursor:
    print(row)

('Atlanta', datetime.datetime(2019, 11, 6, 16, 28, 29, 197000), datetime.date(2019, 11, 6), 56.17, 56.17, 0.0, 5.59)
('Boston', datetime.datetime(2019, 11, 6, 16, 28, 29, 183000), datetime.date(2019, 11, 6), 38.25, 38.25, 0.0, 11.41)
('Boulder', datetime.datetime(2019, 11, 6, 16, 28, 29, 200000), datetime.date(2019, 11, 6), 30.47, 30.47, 0.0, 5.82)
('Chicago', datetime.datetime(2019, 11, 6, 16, 28, 29, 193000), datetime.date(2019, 11, 6), 37.69, 37.69, 0.0, 9.17)
('Denver', datetime.datetime(2019, 11, 6, 16, 28, 29, 200000), datetime.date(2019, 11, 6), 30.36, 30.36, 0.0, 5.82)
('Detroit', datetime.datetime(2019, 11, 6, 16, 28, 29, 190000), datetime.date(2019, 11, 6), 38.01, 38.01, 0.0, 3.36)
('Helena', datetime.datetime(2019, 11, 6, 16, 28, 29, 197000), datetime.date(2019, 11, 6), 27.66, 27.66, 0.0, 3.36)
('Honolulu', datetime.datetime(2019, 11, 6, 16, 28, 29, 197000), datetime.date(2019, 11, 6), 83.71, 83.71, 0.0, 14.99)
('Houston', datetime.datetime(2019, 11, 6, 16, 28, 29, 197000), 

In [49]:
print(values)


('Atlanta', 'Land-Locked', 'GA')


In [50]:
values[0]

'Atlanta'